<a href="https://colab.research.google.com/github/JPxYang/PythonLearn/blob/main/%E6%8E%A8%E8%96%A6%E6%BC%94%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from google.cloud import bigquery
from google.oauth2 import service_account

In [31]:
credentials = service_account.Credentials.from_service_account_file('service-project-data-prod-ffbd7f3273b8.json')


def query_bigquery_table(project_id):
  # 初始化 BigQuery 客戶端
  bq_client = bigquery.Client(credentials=credentials)
  # 構建查詢
  query = f"""
        with postreads as (
              select * from `WellW.post_reads`
        ),
        posts as (
              select * from `WellW.posts`
        ),
        d7 as (
            SELECT  COUNT(pr.read_log_id) AS count ,pr.post_id as article_seq ,pr.post_id as id , p.title
            FROM postreads as pr
            JOIN posts as p  ON pr.post_id = p.id
            WHERE DATE(pr.create_at) >= DATE_ADD( CURRENT_DATE('+08'), INTERVAL -1 DAY)
            GROUP BY pr.post_id, p.title
            ORDER BY count DESC
        ),
        d7x as (
            SELECT *   ,row_number() over (order by count desc) as row_num
            from d7
        ),
        d8 as (
            SELECT  COUNT(pr.read_log_id) AS count ,pr.post_id as article_seq ,pr.post_id as id , p.title
            FROM postreads as pr
            JOIN posts as p  ON pr.post_id = p.id
            WHERE DATE(pr.create_at) >= DATE_ADD( CURRENT_DATE('+08'), INTERVAL -2 DAY)
            and  DATE(pr.create_at) <= DATE_ADD( CURRENT_DATE('+08'), INTERVAL -1 DAY)
            GROUP BY pr.post_id, p.title
            ORDER BY count DESC
        ),
        d8x as (
            SELECT *   ,row_number() over (order by count desc) as row_num
            from d8
        ),
        postlikeCounts as (
            SELECT   count(post_id) as postlikeCount ,post_id
            from `WellW.post_likes`
            group by post_id
        ),
        final as(
        select d7x.id,d7x.title ,d7x.count as row7,ifnull(d8x.count,99) as row8
        ,posts.create_at
        ,plc.postlikeCount
        from d7x left join d8x on d7x.id=d8x.id
        join posts on d7x.id=posts.id
        join postlikeCounts as plc on plc.post_id = d7x.id
        )
        select * from final
       """

  # 執行查詢
  query_job = bq_client.query(query , project=project_id )
  # 取得查詢結果
  results = query_job.result()

  return results

In [32]:
results = query_bigquery_table('t-solstice-405908')
# 將結果轉換為 JSON 格式
data_list = []
for row in results:
  data_list.append(dict(row.items()))

In [38]:
data_list[0]

{'id': '01HNF15TT4GDR9QQW9V8DJBMEX',
 'title': '貓和倉鼠可以一起養嗎',
 'row7': 18,
 'row8': 14,
 'create_at': datetime.datetime(2024, 1, 31, 5, 51, 54, 692870),
 'postlikeCount': 4}

In [5]:
len(data_list)

74

In [34]:
# Rewritten code from /r2/r2/lib/db/_sorts.pyx

from datetime import datetime, timedelta
from math import log

epoch = datetime(1970, 1, 1)

def epoch_seconds(date):
    td = date - epoch
    return td.days * 86400 + td.seconds + (float(td.microseconds) / 1000000)

def score(ups, downs):
    return ups - downs

def hot(ups, downs, date):
    s = score(ups, downs)
    order = log(max(abs(s), 1), 2)
    sign = 1 if s > 0 else -1 if s < 0 else 0
    seconds = epoch_seconds(date) - 1134028003
    return round(sign * order + seconds / 45000, 7)

In [39]:
import math
def _confidence(ups,downs):
    """The confidence sort.
       http://www.evanmiller.org/how-not-to-sort-by-average-rating.html"""
    n = ups + downs

    if n == 0:
      return 0

    z = 1.281551565545 # 80% confidence
    p = float(ups) / n

    left = p + 1/(2*n)*z*z
    right = z * math.sqrt(p*(1-p)/n + z*z/(4*n*n))
    under = 1+1/n*z*z

    return (left - right) / under

In [41]:
a1 = hot(1,1,datetime(2024, 1, 25) )
a2 = _confidence(10,0)
a1,a2

(12713.6177111, 0.8589313179093836)

In [ ]:
import pandas as pd
Postdata_list = []
for PostData in data_list:
  print( hot(PostData["row7"] , 0 ,PostData["create_at"] ) ,PostData["id"] ,PostData["title"])
  listX = []
  listX.append(hot(PostData["row7"] , 0 ,PostData["create_at"] ))
  listX.append(PostData["id"])
  listX.append(PostData["title"])
  listX.append(PostData["create_at"])
  listX.append(PostData["row7"])
  Postdata_list.append(listX)

In [36]:
def takeSecond(elem):
  return elem[0]

In [37]:
Postdata_list.sort(key=takeSecond, reverse=True)
Postdata_list

[[12731.0521239,
  '01HNF0J998DQPM0VXM7HGGFCRZ',
  '誰說運動一定要去健身房？過年就把遊戲籌碼(錢)換成運動組數吧！',
  datetime.datetime(2024, 1, 31, 5, 41, 14, 152048),
  44],
 [12730.2999806,
  '01HNF39CTRFAY3N4T9K0WZMBTX',
  '臥虎藏「龍」🐲到森林遊樂區走春探險',
  datetime.datetime(2024, 1, 31, 6, 28, 48, 600035),
  25],
 [12730.0753141,
  '01HNF7EZ0WX90K0K4W8RVFN0M9',
  '2023幸福企業出爐啦~~',
  datetime.datetime(2024, 1, 31, 7, 41, 45, 372079),
  20],
 [12730.0179579,
  '01HNESPZBJ7GEBV6PJDD4A49J9',
  '#WellW活動  發文按讚留言就能拿獎金？',
  datetime.datetime(2024, 1, 31, 3, 41, 27, 794548),
  24],
 [12729.994623,
  '01HNERPXWSGEYKTKPEEX3282CA',
  '你知道包裝食品上的素食標示差別在哪嗎？',
  datetime.datetime(2024, 1, 31, 3, 23, 57, 721521),
  24],
 [12729.8084152,
  '01HNEZ621JYQ9P3F965NFCWRRF',
  '請假提前返鄉好嗎?',
  datetime.datetime(2024, 1, 31, 5, 17, 4, 946721),
  19],
 [12729.7768515,
  '01HNF15TT4GDR9QQW9V8DJBMEX',
  '貓和倉鼠可以一起養嗎',
  datetime.datetime(2024, 1, 31, 5, 51, 54, 692870),
  18],
 [12729.7564796,
  '01HNCPSSMBWAA91FESYBB71F0A',
  '打牌、追劇、零食一把一把抓，過年點心健康3招吃起